In [47]:
from openpyxl import load_workbook

checklist = load_workbook('sample.xlsx')
active_sheet = checklist.active


In [48]:
a1 = active_sheet['A1']


In [49]:
a1.value

'Field'

In [50]:
data = [active_sheet.cell(row=2,column=i).value for i in range(2,7) ]

In [51]:
data

[2, 3, 1, None, 3]

In [59]:
non_empty_data = filter(None, data)
for elem in non_empty_data:
    print(elem)
    
    

2
3
1
3


In [54]:
sum(non_empty_data)

9

## Intentando algo un poco mas avanzado y modular

In [245]:
# This needs `pip install openpyxl` or `conda install openpyxl`
from openpyxl import load_workbook

In [246]:
def get_active_sheet(filename):
    '''
    This function reads an xlsx file in the same folder
    as this notebook and returns a variable containing
    the active sheet.
    '''
    workbook = load_workbook(filename)
    active_sheet = workbook.active
    return active_sheet

In [247]:
def get_weekly_data(sheet):
    '''
    Given a sheet variable containing a data in the form of:
    | Field   | Mon | Tue | Wed | Thu | Fri |
    -----------------------------------------
    | field 1 | 1   |     |     | 7   | 2   |
    | field 2 |     |     |     | 1   |     |
    | field 3 | 2   |     | 5   | 1   |     |
    return a dictionary where the keys are the field names and 
    the value is a list of 5 element containing the cell values 
    per week day, e.g.:
    
    result = {
        "field 1": [1,    None, None, 7, 2   ],
        "field 2": [None, None, None, 1, None],
        "field 3": [2,    None, 5,    1, None]
    
    }
    '''
    # We'll save the dictionary of lists in the `all_rows` variable,
    # initally empty
    all_rows = {}
    # We don't know the number of rows in advance, so we iterate
    # from the 2nd (the 1st one is just the column titles), to the
    # last one given by `sheet.max_row`.
    for row in range(2, sheet.max_row + 1): # `+1` is needed because range excludes the last index
        row_list = [] # empty list to store contents of current row
        # We know data is in columns Mon-Fri, with index 2 to 
        # 6 (inclusive of 6).
        field_name = sheet.cell(row=row, column=1).value
        for column in range(2, 7):
            current_cell = sheet.cell(row=row, column=column)
            row_list.append(current_cell.value)
        all_rows[field_name] = row_list
    return all_rows
        

In [264]:
def clean_data(messy_dict):
    '''
    Receives a dictionary of lists which contain some empty 
    (`None`) values, and returns a similar dictionary with
    clean lists with the None values filtered out.
    E.g. given:
    
    messy_dict = {
        "field 1": [1,    None, None, 7, 2   ],
        "field 2": [None, None, None, 1, None],
        "field 3": [2,    None, 5,    1, None]
    }
    
    it returns:
    
    clean_dict = {
        "field 1": [1, 7, 2],
        "field 2": [1],
        "field 3": [2, 5, 1]
    }    
    '''
    clean_dict = {}
    for key in messy_dict.keys():
        filtered_list = filter(None, messy_dict[key])
        clean_dict[key] = filtered_list
    return clean_dict
        

In [273]:
def sum_data(clean_dict):
    '''
    Receives a list of lists with numbers and returns a list of sum results per list.
    E.g.     
    
    clean_dict = {
        "field 1": [1, 7, 2],
        "field 2": [1],
        "field 3": [2, 5, 1]
    }    
             
    returns:
             
    sum_dict = {
        "field 1": 10,
        "field 2": 1,
        "field 3": 8
    }  
    
    '''
    sum_dict = {}
    for key in clean_dict.keys():
        num_sum = sum(clean_dict[key])
        sum_dict[key] = num_sum
    return sum_dict

In [274]:
def write_sum_list(sum_dict, output_filename, column):
    '''
    Receives a sum dictionary, e.g.: {
                                    "field 1": 10, 
                                    "field 2": 1, 
                                    "field 3": 8
                                     }
    and saves it on a given excel file in `output_filename`,
    the cell in the given column next to each field.
    
    E.g. the input excel might contain:

    | Field   | Sum |
    -----------------
    | field 1 |     |
    | field 2 |     |
    | field 3 |     |
    
    And we'll update it like this:
    
    | Field   | Sum |
    -----------------
    | field 1 | 10  |
    | field 2 | 1   |
    | field 3 | 8   |   
    
    '''
    workbook = load_workbook(output_filename)
    sheet = workbook.active
    for key in sum_dict:
        # Find key in output workbook
        for row in range(1, sheet.max_row + 1):
            if (sheet.cell(row=row, column=column).value == key):
                sheet.cell(row=row, column=column+1).value = sum_dict[key]
    workbook.save(output_filename)
    

Let's start: read the file called `checklist.xslx` in current directory and save the active sheet on a variable so we can refer to it later on.

In [275]:
sheet = get_active_sheet('checklist.xlsx')

Now load the data within the sheet and save to dictionary where the keys are the checklists' field names and the values are a list of numbers representing incidences from Mon to Fri.

In [276]:
checklist_data = get_weekly_data(sheet)

In [277]:
checklist_data

{'field 1': [1, None, None, 7, 2],
 'field 2': [None, None, None, 1, None],
 'field 3': [2, None, 5, 1, None]}

We can't make sums with `None` values, so let's get rid of them:

In [278]:
clean_checklist = clean_data(checklist_data)

Now we are ready to sum the data for each field:

In [279]:
sums = sum_data(clean_checklist)

In [280]:
sums

{'field 1': 10, 'field 2': 1, 'field 3': 8}

Let's save it on a different file:

In [281]:
write_sum_list(sums, 'output.xlsx', 1)